In [ ]:
import pandas as pd

responses_data = pd.read_csv("./Connection_to_Nature_Data.csv")
responses_data.head()

In [ ]:
questions_data = pd.read_csv("questions.csv", delimiter=";")
questions_data.head()

In [ ]:
# Preprocess participants data
participants_df = pd.DataFrame({
    "ParticipantID": range(1, len(responses_data) + 1),
    "Age": responses_data["D_Age"],
    "Gender": responses_data["D_Gender"].map({"1": "Male", "2": "Female"}),  # Map gender codes
    "Nationality": responses_data["D_Nation"],
    "HoursSpentInNature": responses_data["D_hours"]
})

# Export table to CSV
participants_df.to_csv("Participants.csv", index=False)

In [ ]:
# Preprocess activities data
# Handle missing or invalid data in D_activities
responses_data["D_activities"] = responses_data["D_activities"].fillna("").astype(str)

# Extract unique activities from the dataset
unique_activities = set(
    activity.strip().lower() for activities in responses_data["D_activities"] for activity in activities.split(",") if activity.strip()
)

# Create the Activities table with consistent formatting
activities_df = pd.DataFrame({
    "ActivityID": range(1, len(unique_activities) + 1),
    "ActivityName": list(unique_activities)
})

# Map Participants to Activities
participant_activities = []
for participant_id, activities in enumerate(responses_data["D_activities"], start=1):
    for activity in activities.split(","):
        activity_name = activity.strip().lower()
        # Ensure the activity exists in the Activities dataframe
        matching_activity = activities_df.loc[activities_df["ActivityName"] == activity_name]
        if not matching_activity.empty:
            activity_id = matching_activity["ActivityID"].values[0]
            participant_activities.append({"ParticipantID": participant_id, "ActivityID": activity_id})
        else:
            print(f"Activity '{activity_name}' not found in Activities table.")

# Create the Participants_Activities table
participants_activities_df = pd.DataFrame(participant_activities)

# Export tables to CSV
activities_df.to_csv("Activities.csv", index=False)
participants_activities_df.to_csv("Participants_Activities.csv", index=False)


In [ ]:
# Preprocess allotment participation data
allotment_participation_df = pd.DataFrame({
    "AllotmentID": range(1, len(responses_data) + 1),
    "ParticipantID": range(1, len(responses_data) + 1),
    "OwnsAllotment": responses_data["D_allot"].map({1: "1", 2: "0"}),
    "ParticipatesInGroup": responses_data["D_group"].map({1: "1", 2: "0"})
})

# Export table to CSV
allotment_participation_df.to_csv("AllotmentParticipation.csv", index=False)

In [ ]:
# Preprocess answer types data
# Extract unique answer types from the `Values` column
answer_types_df = questions_data[["Values"]].drop_duplicates().reset_index(drop=True)
answer_types_df["AnswerTypeID"] = range(1, len(answer_types_df) + 1)
answer_types_df = answer_types_df.rename(columns={"Values": "AnswerOptions"})
answer_types_df["ScaleType"] = "Likert"
answer_types_df["Description"] = "Extracted scale type from provided values"

# Export table to CSV
answer_types_df.to_csv("AnswerTypes.csv", index=False)

In [ ]:
# Check the largest value size in AnswerTypes.AnswerOptions
max_length_answer_options = answer_types_df['AnswerOptions'].astype(str).map(len).max()
print(f"The largest value size in AnswerTypes.AnswerOptions is: {max_length_answer_options} characters")

In [ ]:
# Preprocess questions data
# Map AnswerTypeID to the Questions table
questions_data = questions_data.merge(answer_types_df, left_on="Values", right_on="AnswerOptions")
questions_data["QuestionID"] = range(1, len(questions_data) + 1)

# Select and reorder columns for the questions Table
questions_df = questions_data[["QuestionID", "Section title", "Code", "Item", "AnswerTypeID"]].rename(columns={
    "Section title": "Section",
    "Code": "Code",
    "Item": "Item"
})

# Export table to CSV files
questions_df.to_csv("Questions.csv", index=False)

In [ ]:
# Check the largest value size in Questions.Item
max_length = questions_df["Item"].astype(str).apply(len).max()
print(f"The largest value in 'Questions.Item' has {max_length} characters.")

In [ ]:
# 7. Preprocess survey responses data
survey_responses = []
for participant_id, row in responses_data.iterrows():
    for question_code in questions_df["Code"]:
        if question_code in row:
            survey_responses.append({
                "ParticipantID": participant_id + 1,
                "QuestionID": questions_df.loc[questions_df["Code"] == question_code, "QuestionID"].values[0],
                "AnswerValue": row[question_code]
            })
survey_responses_df = pd.DataFrame(survey_responses)

# Export table to CSV files
survey_responses_df.to_csv("SurveyResponses.csv", index=False)